# Music Generated With LSTMs

In this notebook we implement a model able to generate music from an input in ABC format. This music notation uses only ASCII chars, so it can be easily fed into RNNs. You can find details and examples of ABC notation here: http://abcnotation.com/.

The dataset we'll be using is the [ABC version of the Nottingham Music Database](http://abc.sourceforge.net/NMD/).

## Get dataset

In [ ]:
import json
import pickle
from pathlib import Path

from core.input import get_abcs, get_dataset


VERSION = 'notebook'

model_dir = Path('./model-data/{}'.format(VERSION))
model_dir.mkdir(parents=True, exist_ok=True)

abc_sources = [
    'http://abc.sourceforge.net/NMD/nmd/jigs.txt',
    'http://abc.sourceforge.net/NMD/nmd/hpps.txt',
    'http://abc.sourceforge.net/NMD/nmd/morris.txt',
    'http://abc.sourceforge.net/NMD/nmd/playford.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsa-c.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsd-g.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsh-l.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsm-q.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsr-t.txt',
    'http://abc.sourceforge.net/NMD/nmd/reelsu-z.txt',
    'http://abc.sourceforge.net/NMD/nmd/slip.txt',
    'http://abc.sourceforge.net/NMD/nmd/waltzes.txt',
    'http://abc.sourceforge.net/NMD/nmd/xmas.txt',
    'http://abc.sourceforge.net/NMD/nmd/ashover.txt',
]

abcs = get_abcs(abc_sources)
dataset, token2id = get_dataset(abcs, remove_comments=True)

with (model_dir/'dataset.pickle').open('wb') as f:
    pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

with (model_dir/'token2id.json').open('w') as f:
    json.dump(token2id, f)

In [ ]:
import pathlib
import random

import pretty_midi
from IPython.display import Audio, display


for abc in random.sample(abcs, 3):
    filename = 'temp'
    with open(filename, 'w') as f:
        f.write(abc)
    !/abcmidi/abc2midi.exe {filename} > /dev/null
    midi = pretty_midi.PrettyMIDI(f'{filename}1.mid')
    display(Audio(midi.fluidsynth(), rate=44100))

pathlib.Path(filename).unlink()

## Training time

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from core.model import grid_search

param_grid = dict(
    num_unique_chars=[len(token2id)],
    batch_size=[64],
    seq_length=[96],
    embedding_dim=[64, 128],
    rnn_units=[[256, 256], [128, 128, 128]],
)

grid_search(
    dataset,
    model_dir,
    param_grid,
    epochs=50,
)

"""
best result:
    embedding_dim=[128],
    rnn_units=[[128, 128, 128]],

overfit:
    rnn_units=[[256, 256]],
"""

In [ ]:
from core.model import grid_search

param_grid = dict(
    num_unique_chars=[len(token2id)],
    batch_size=[64],
    seq_length=[96],
    embedding_dim=[128],
    rnn_units=[[128, 128, 128]],
    recurrent_dropout=[0.05, 0.3],
    dropout=[0.05, 0.3],
)

grid_search(
    dataset,
    model_dir,
    param_grid,
    epochs=100,
    initial_epoch=50,
    weights_to_load='../batch_size=64,seq_length=96,embedding_dim=128,rnn_units=[128, 128, 128]/weights-050-0.6451-0.6392.h5'
)

In [ ]:
from core.model import train_model

params = dict(
    num_unique_chars=len(token2id),
    batch_size=64,
    seq_length=96,
    embedding_dim=128,
    rnn_units=[128, 128, 128],
    recurrent_dropout=0.05,
    dropout=0.3,
)

train_model(
    dataset,
    model_dir,
    params,
    epochs=100,
)

## Results

Let's take a look at the accuracy and loss during training.

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv(model_dir/'log.csv')
fig = px.line()
fig.add_scatter(x=df.epoch, y=df.acc, mode='lines', name='acc')
fig.add_scatter(x=df.epoch, y=df.val_acc, mode='lines', name='val_acc')
fig.show()

In [ ]:
fig = px.line()
fig.add_scatter(x=df.epoch, y=df.loss, mode='lines', name='loss')
fig.add_scatter(x=df.epoch, y=df.val_loss, mode='lines', name='val_loss')
fig.show()

## Enjoy music!

In [ ]:
from core.model import generate_sequence

all_weights =  sorted(*[model_dir.rglob('*.h5')])
weights_to_load = str(all_weights[-1]).split('/')[-1]
seq = generate_sequence(model_dir, weights_to_load, output_length=1500)
abcs = seq.split('X: 1\n')
abcs = '\n'.join('X: {}\n{}\n'.format(i+1, abc) for i, abc in enumerate(abcs[1:]))

In [ ]:
with open('abc', 'w') as f:
    f.write(abcs)

!/abcmidi/abc2midi.exe abc > /dev/null

In [ ]:
import pathlib

import pretty_midi
from IPython.display import Audio, display


path = pathlib.Path('.')
files = [str(filename) for filename in path.rglob('*.mid')]

for f in files:
    midi = pretty_midi.PrettyMIDI(f)
    display(Audio(midi.fluidsynth(), rate=44100))